<a href="https://colab.research.google.com/github/Maplemx/Agently/blob/main/playground/human_step_in_before_reply.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Human Step In Before Reply

## Demo Description

**Author:** Agently Team

**Prompt Language:** English

**Agent Components:** EventListener, ReplyReformer

**Description:**

Sometimes we don't want the AI agent's response to be the final one directly to user. Instead, we want to have the opportunity to review the response and confirm it before it is sent to user. Or even, sometimes we hope to modify the reply.

The following case demonstrates how to do that using Agently framework easily. We can pause the agent's output, undergo human review and verification, modify the reply if it is necessary. Then we can append the new reply into the chat history list.

Through this case, we also can observe how AI agent dynamically adjusts its behavior based on the modified reply.

This is just a simple example. You can extend upon this by, for examples, using rule system instead of human verification, using this method to build your chatting samples in real time interaction with language model for model fine-tuning or just storing the samples into RoleManager (Read [Agently Application Development Handbook: RoleManager](https://github.com/Maplemx/Agently/blob/main/docs/guidebook/application_development_handbook.ipynb) to explore more).

在一些场景下，我们并不希望AI Agent的回复就是最终回复，而是希望我们有机会在AI Agent给出回复前进行审查确认，甚至是介入修改回复。

下面的案例给出了如何使用Agently完成这个过程。案例展现了如何暂停Agent输出，人工审核确认或是修订后再进行返回，并将返回结果写入对话记录的全过程。

通过这个案例，我们也看到AI Agent在过程中根据修订结果，即时调整了自己的行为反馈。

这只是一个简单的案例，你可以基于这个案例做进一步的扩展，例如：使用其他规则系统替代人工审核确认，使用这种交互方式训练你自己的对话样本集用于模型微调或是存入角色库等。

关于角色库：想要了解更多，可以阅读[📖 Agently应用开发手册：RoleManager章节](https://github.com/Maplemx/Agently/blob/main/docs/guidebook/application_development_handbook.ipynb)

## Step 1: Install Packages

In [ ]:
!pip install Agently

## Step 2: Demo Code

In [1]:
import Agently

agent_factory = Agently.AgentFactory()
agent_factory\
    .set_settings("model.OpenAI.auth", { "api_key": "" })

agent = agent_factory.create_agent()
# We need to manage chat history manually in this case because
# we want to change reply.
# Agently Team To-Do:
# - We will update Session component to support this use case
#   to record chat history with edited reply automatically
chat_history = []
while True:
    # Confirm reply or edit it
    def human_confirm(data):
        global edited_reply
        confirm = ""
        while confirm not in ("y", "n"):
            confirm = input("\nDo you confirm? [Y/N]: ").lower()
        if confirm == "n":
            while edited_reply == "":
                edited_reply = input("Correct reply: ")
    # Use ReplyReformer component to change reply
    def change_reply(data):
        if edited_reply == "":
            return data["reply"]
        else:
            return edited_reply

    # Start main process
    edited_reply = ""
    global chat_history
    user_input = input("[USER]: ")
    if user_input == "#exit":
        break
    print("[Assistant] want to response:")
    # - on_delta: print response in realtime
    # - on_done: pause output process and call `human_confirm()`
    # - reform_reply: call `change_reply()` to edit final return
    reply = agent\
        .chat_history(chat_history)\
        .input(user_input)\
        .on_delta(lambda data: print(data, end=""))\
        .on_done(human_confirm)\
        .reform_reply(change_reply)\
        .start()
    print("[Assistant]: ", reply)
    chat_history.extend([
        { "role": "user", "content": user_input },
        { "role": "assistant", "content": reply }
    ])
print("------------\n", "[Chat History]: \n", chat_history)

[USER]: 你好
[Assistant] want to response:
你好！有什么我可以帮助你的吗？
Do you confirm? [Y/N]: n
Correct reply: 哦...
[Assistant]:  哦...
[USER]: 怎么了？心情不好吗？
[Assistant] want to response:
不，我只是觉得你没有提供足够的信息，所以不知道该如何回答你。如果你有什么问题或者需要帮助，请随时告诉我。
Do you confirm? [Y/N]: n
Correct reply: 对，我昨天晚上没有睡好
[Assistant]:  对，我昨天晚上没有睡好
[USER]: 啊？严重吗？要不要去医院看看？
[Assistant] want to response:
不用这么严重，只是睡眠不够充足而已。可能是工作压力比较大导致的。我只需要多休息一下就好了。谢谢你的关心。
Do you confirm? [Y/N]: y
[Assistant]:  不用这么严重，只是睡眠不够充足而已。可能是工作压力比较大导致的。我只需要多休息一下就好了。谢谢你的关心。
[USER]: #exit
------------
 [Chat History]: 
 [{'role': 'user', 'content': '你好'}, {'role': 'assistant', 'content': '哦...'}, {'role': 'user', 'content': '怎么了？心情不好吗？'}, {'role': 'assistant', 'content': '对，我昨天晚上没有睡好'}, {'role': 'user', 'content': '啊？严重吗？要不要去医院看看？'}, {'role': 'assistant', 'content': '不用这么严重，只是睡眠不够充足而已。可能是工作压力比较大导致的。我只需要多休息一下就好了。谢谢你的关心。'}]


---

[**_<font color = "red">Agent</font><font color = "blue">ly</font>_** Framework - Speed up your AI Agent Native application development](https://github.com/Maplemx/Agently)